In [2]:
# Surpress warnings
import warnings
warnings.filterwarnings("ignore")

# Import libraries
from pmlb import dataset_names, classification_dataset_names, fetch_data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

# Import SK-learn and AutoSK-Learn
import autosklearn.classification
import autosklearn.regression
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

/home/jcui/.conda/envs/auto-sklearn/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [3]:
usable_datasets = []
dataset_min_count = 10
dataset_max_count = 20
count = 0

for dataset in classification_dataset_names:
    if count < dataset_min_count:
        count += 1
        continue
    usable_datasets.append(dataset)
    count += 1
    if count >= dataset_max_count:
        break


In [4]:
usable_datasets

['allbp',
 'allhyper',
 'allhypo',
 'allrep',
 'analcatdata_aids',
 'analcatdata_asbestos',
 'analcatdata_authorship',
 'analcatdata_bankruptcy',
 'analcatdata_boxing1',
 'analcatdata_boxing2']

In [5]:
np.random.seed(0)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

normal_logit_scores = []
junk_logit_scores = []
naive_ens_logit_scores = []
automl_scores = []

for dataset in usable_datasets:
    print("")
    print("NEW DATASET")
    X, y = fetch_data(dataset, return_X_y=True)
    
    m, d = X.shape
    
    # Create and append junk data of feature size 200 (half discrete, half contintinous)
    junk_discrete = np.round(np.random.rand(m, 100))
    junk_continuous = np.random.rand(m, 100)
    junk = np.hstack((junk_discrete, junk_continuous))
    X_junk = np.hstack((X, junk))    
    
    logit_norm = LogisticRegression()
    logit_junk = LogisticRegression()
    
    # Run models on the normal data and assess performance with regular ML models
    train_X_reg, test_X_reg, train_y_reg, test_y_reg = train_test_split(X, y)
    logit_norm.fit(train_X_reg, train_y_reg)
    normal_logit_score = logit_norm.score(test_X_reg, test_y_reg)
    normal_logit_scores.append(normal_logit_score)
    print("Score Logit Normal: ", normal_logit_score)
    
    # Run models on the normal+junk data and assess performance with regular ML models
    train_X_junk, test_X_junk, train_y_junk, test_y_junk = train_test_split(X_junk, y)
    logit_junk.fit(train_X_junk, train_y_junk)
    junk_logit_score = logit_junk.score(test_X_junk, test_y_junk)
    junk_logit_scores.append(junk_logit_score)
    print("Score Logit Junk: ", junk_logit_score)
    
    # Run naive ensembling with naive average with regular ML models
    train_X_reg, test_X_reg, train_y, test_y = train_test_split(X, y, random_state=0)
    train_X_junk, test_X_junk, train_y, test_y = train_test_split(junk, y, random_state=0)
    
    logit_p1 = LogisticRegression()
    logit_p2 = LogisticRegression()
    
    logit_p1.fit(train_X_reg, train_y)
    logit_p2.fit(train_X_junk, train_y)    
    
    pred1 = logit_p1.predict(test_X_reg)
    pred2 = logit_p2.predict(test_X_junk)
    
    pred_y = np.round(0.5 * pred1 + 0.5 * pred2)
    naive_ens_score = accuracy_score(pred_y, test_y)
    print("Naive Ensemble: ", naive_ens_score)
    naive_ens_logit_scores.append(naive_ens_score)
                    
    # Run Auto-SKLearn on the normal + junk data
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_junk, y)
    automl = autosklearn.classification.AutoSklearnClassifier()
    automl.fit(X_train, y_train)
    y_hat = automl.predict(X_test)
    automl_score = sklearn.metrics.accuracy_score(y_test, y_hat);
    print("AutoML score: ", automl_score)
    automl_scores.append(automl_score)
    

    # TODO: Run Block Regression
    # TODO: Run Block Regression and Auto-SkLearn
    
    # TODO: Repeat this entire process but with different size junk groups and different numbers
    
    
    
    
    


NEW DATASET
Score Logit Normal:  0.9618239660657476
Score Logit Junk:  0.9671261930010604
Naive Ensemble:  0.953340402969247
[WARNING] [2019-02-28 00:21:05,428:EnsembleBuilder(1):e0ce6b46d04303af3ce7eaf24e477974] No models better than random - using Dummy Score!
[WARNING] [2019-02-28 00:21:05,441:EnsembleBuilder(1):e0ce6b46d04303af3ce7eaf24e477974] No models better than random - using Dummy Score!
[WARNING] [2019-02-28 00:21:06,044:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-02-28 00:21:06,044:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-02-28 00:21:07,452:EnsembleBuilder(1):e0ce6b46d04303af3ce7eaf24e477974] No models better than random - using Dummy Score!


In [9]:
print("Average score for normal logit: ", str(sum(normal_logit_scores)/float(len(normal_logit_scores))))
print("Average score for normal+junk logit: ", str(sum(junk_logit_scores)/float(len(junk_logit_scores))))
print("Average score for naive ensemble with logit: ", str(sum(naive_ens_logit_scores)/float(len(naive_ens_logit_scores))))
print("Average score for automl for normal+junk: ", str(sum(automl_scores)/float(len(automl_scores))))

Average score for normal logit:  0.849080506924536
Average score for normal+junk logit:  0.8195549985171189
Average score for naive ensemble with logit:  0.7824321681377665
Average score for automl for normal+junk:  0.8349368658912667


In [14]:
X_junk

array([[1.        , 0.        , 0.        , ..., 0.40292058, 0.31297249,
        0.4414766 ],
       [0.        , 1.        , 0.        , ..., 0.12702129, 0.8205428 ,
        0.86351832],
       [0.        , 0.        , 0.        , ..., 0.71576606, 0.58220464,
        0.57942425],
       ...,
       [2.        , 0.        , 0.        , ..., 0.47287437, 0.01392644,
        0.63438904],
       [1.        , 0.        , 0.        , ..., 0.42546666, 0.67785646,
        0.18979169],
       [1.        , 0.        , 0.        , ..., 0.78412312, 0.61925482,
        0.15752893]])

array([0, 0, 0, ..., 0, 0, 0])